In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch import optim
from torch.nn import Embedding
from torch.autograd import Variable
import torch.nn.functional as F


In [2]:
# train dataset
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [3]:
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2])
          for i in range(len(test_sentence)-2)]

In [4]:
# 将每个单词编码
vocb = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [5]:
#define N Gram
class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        self.embedding = Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size*n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)
        
    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        log_prob = F.log_softmax(out, dim=1)
        return log_prob

In [6]:
ngram = NgramModel(len(vocb), CONTEXT_SIZE, EMBEDDING_DIM)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ngram.parameters(), lr=1e-2, weight_decay=1e-5)


In [7]:
epoch = 200
for e in range(epoch):
    train_loss = 0
    for word, label in trigram:
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        output = ngram(word)
        loss = criterion(output, label)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(train_loss/len(trigram))       

4.987432937706466
3.3540546731875005
1.0814985939885173
0.3864251757067341
0.09810662657415313
0.0555082476625167
0.049916519789795666
0.0467532854375277
0.04533410951977863
0.05223839276770812
0.058881534539784855
0.04718340065334967
0.04233097136902146
0.040502370365134766
0.04267641846746679
0.040838821730812117
0.041138941299137935
0.050163046100576855
0.07570814660544156
0.04283310641323173
0.04976670081089498
0.04487244128411846
0.03754658735594434
0.04062092941062756
0.03921988525580351
0.03743961033924118
0.03799236347659056
0.038288960620952536
0.0363403348315536
0.03964861650514392
0.03705331598461577
0.0376815433317509
0.03891293756639074
0.042401417272898936
0.05571716320869643
0.13054617703042357
0.401549558211579
3.206679124232561
1.7059362919301866
0.0654851120593001
0.040582053151880536
0.03798523969704323
0.036732343356939645
0.03593573503259997
0.03540064340770883
0.03502691860275169
0.0347672845844368
0.03459669668803035
0.03600070457723593
0.03453681051609876
0.0343

In [8]:
word, label = trigram[11]
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = ngram(word)
_, predict = torch.max(out, 1)
predict_word = idx_to_word[predict.item()]
print('real word is {}, predict word is {}'.format(label, predict_word))

real word is beauty's, predict word is beauty's
